In [ ]:
import csv
import datetime  # Importing datetime module


class Book:
    """
    The Book class represents books in the library system.
    """
    def __init__(self, title, author, isbn, status="Available", borrower="", genre="Unknown", reading_time=0):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.status = status
        self.borrower = borrower
        self.genre = genre
        self.reading_time = reading_time

    def toString(self):
        return f"{self.title},{self.author},{self.isbn},{self.status},{self.borrower},{self.genre},{self.reading_time}"


class LibrarySystem:
    """
    The LibrarySystem class is the main interface for library operations.
    """
    def __init__(self):
        self.BookList = []
        self.user_activity = {}

    def start(self):
        """
        Loads the library data from a CSV file, skipping the header if present.
        """
        try:
            with open("library.csv", "r", newline='', encoding="utf-8") as file:
                reader = csv.reader(file)
                self.BookList.clear()
                next(reader, None)  # Skip header row
                for row in reader:
                    if len(row) == 7:
                        try:
                            book = Book(row[0], row[1], row[2], row[3], row[4], row[5], int(row[6]))
                            self.BookList.append(book)
                        except ValueError as e:
                            print(f"Skipping record due to value error: {row}. Error: {e}")
                    else:
                        print(f"Skipping malformed record: {row}")
            print("\nCity Library System is now running.\n")
        except FileNotFoundError:
            print("The library database file is missing. Starting with an empty library.")
            self.BookList.clear()

    def exit(self):
        """
        Saves the library data to a CSV file when the system exits.
        """
        print(f"\n{self.getCurrentTime()} - Closing the City Library System.\n")
        try:
            with open("library.csv", "w", newline='', encoding="utf-8") as file:
                writer = csv.writer(file)
                writer.writerow(["Title", "Author", "ISBN", "Status", "Borrower", "Genre", "Reading Time"])
                for book in self.BookList:
                    writer.writerow([book.title, book.author, book.isbn, book.status, book.borrower, book.genre, book.reading_time])
        except Exception as e:
            print(f"Failed to write to library_data.csv: {e}")

    def addBook(self, book):
        print(f"{self.getCurrentTime()} - Adding book '{book.title}'.")
        self.BookList.append(book)
        print(f"Book '{book.title}' added successfully.")

    def borrowBook(self, isbn, borrower_name):
        print(f"{self.getCurrentTime()} - Borrowing attempt for ISBN {isbn} by {borrower_name}.")
        book = self.getBookByISBN(isbn)
        if book:
            if book.status == "Available":
                book.status = "Borrowed"
                book.borrower = borrower_name
                print(f"Book '{book.title}' is successfully borrowed by {borrower_name}.")
                self.user_activity[borrower_name] = self.user_activity.get(borrower_name, 0) + 1
            else:
                print(f"Book '{book.title}' is already borrowed by {book.borrower}.")
        else:
            print(f"No book found with ISBN {isbn}.")

    def returnBook(self, isbn):
        print(f"{self.getCurrentTime()} - Return attempt for ISBN {isbn}.")
        book = self.getBookByISBN(isbn)
        if book:
            if book.status == "Borrowed":
                book.status = "Available"
                borrower = book.borrower
                book.borrower = ""
                print(f"Book '{book.title}' is successfully returned by {borrower}.")
            else:
                print(f"Book '{book.title}' is already available.")
        else:
            print(f"No book found with ISBN {isbn}.")

    def displayBooks(self, page_number=1, books_per_page=5, filter_by=None, keyword=None):
        print(f"{self.getCurrentTime()} - Displaying books.")
        self.paginateBooks(self.BookList, page_number, books_per_page, filter_by, keyword)

    def displayAvailableBooks(self, page_number=1, books_per_page=5):
        print(f"{self.getCurrentTime()} - Displaying available books.")
        available_books = [book for book in self.BookList if book.status.lower() == "available"]
        self.paginateBooks(available_books, page_number, books_per_page)

    def paginateBooks(self, books, page_number, books_per_page, filter_by=None, keyword=None):
        total_books = len(books)
        total_pages = (total_books + books_per_page - 1) // books_per_page  # Round up to get total pages

        if page_number < 1 or page_number > total_pages:
            print(f"Invalid page number. Please select a page between 1 and {total_pages}.")
            return

        start_index = (page_number - 1) * books_per_page
        end_index = min(start_index + books_per_page, total_books)

        print(f"\nDisplaying page {page_number} of {total_pages}:")
        for i in range(start_index, end_index):
            book = books[i]
            if filter_by and keyword:
                if getattr(book, filter_by).lower() == keyword.lower():
                    print(f"{i + 1}. {book.toString()}")
            else:
                print(f"{i + 1}. {book.toString()}")

        print(f"\nPage {page_number} of {total_pages}")
        if page_number > 1:
            print(f"Previous Page: {page_number - 1}")
        if page_number < total_pages:
            print(f"Next Page: {page_number + 1}")

    def score(self):
        print(f"{self.getCurrentTime()} - Displaying top users based on borrowing activity.")
        if not self.user_activity:
            print("\nNo borrowing activity recorded.")
            return
        
        top_users = sorted(self.user_activity.items(), key=lambda x: x[1], reverse=True)
        
        print("\nTop Users Based on Borrowing Activity:")
        for user, activity in top_users:
            print(f"{user}: {activity} borrows")

    def getBookByISBN(self, isbn):
        """
        Returns a book by its ISBN or None if not found.
        """
        for book in self.BookList:
            if book.isbn == isbn:
                return book
        return None

    def getCurrentTime(self):
        """
        Returns the current date and time in a readable format.
        """
        current_time = datetime.datetime.now()
        return current_time.strftime("%Y-%m-%d %H:%M:%S")  # Format the time as "YYYY-MM-DD HH:MM:SS"


def system():
    library = LibrarySystem()
    library.start()

    while True:
        print("\nWelcome to City Library System!")
        print("1. Borrow Books")
        print("2. Return Books")
        print("3. Add Books")
        print("4. Display Available Books")
        print("5. Search For Books")
        print("6. View Top Users")
        print("7. Exit System")

        choice = input("How can we help? Please enter the number: ")

        if choice == "1":
            borrower_name = input("Please enter your name: ")
            isbn = input("Please enter the ISBN of the book you would like to borrow: ")
            library.borrowBook(isbn, borrower_name)
        elif choice == "2":
            isbn = input("Please enter the ISBN of the book you would like to return: ")
            library.returnBook(isbn)
        elif choice == "3":
            title = input("Enter the title of the book: ")
            author = input("Enter the author of the book: ")
            isbn = input("Enter the ISBN of the book: ")
            genre = input("Enter the genre of the book: ")
            while True:
                try:
                    reading_time = int(input("Enter the estimated reading time (in hours): "))
                    break
                except ValueError:
                    print("Please enter a valid number for reading time.")
            new_book = Book(title, author, isbn, "Available", "", genre, reading_time)
            library.addBook(new_book)
        elif choice == "4":
            page_number = int(input("Enter page number to display: "))
            library.displayAvailableBooks(page_number=page_number, books_per_page=5)
        elif choice == "5":
            filter_by = input("You would like to search by (title/author/status/genre): ")
            keyword = input("Enter the keyword: ")
            library.displayBooks(filter_by=filter_by, keyword=keyword)
        elif choice == "6":
            library.score()
        elif choice == "7":
            library.exit()
            print("\nThank you for using City Library System. See you again!")
            break
        else:
            print("\nInvalid input. Please try again.")


system()



City Library System is now running.


Welcome to City Library System!
1. Borrow Books
2. Return Books
3. Add Books
4. Display Available Books
5. Search For Books
6. View Top Users
7. Exit System


How can we help? Please enter the number:  1
Please enter your name:  Kelly
Please enter the ISBN of the book you would like to borrow:  2233445566


2024-12-11 21:37:23 - Borrowing attempt for ISBN 2233445566 by Kelly.
Book 'War and Peace' is successfully borrowed by Kelly.

Welcome to City Library System!
1. Borrow Books
2. Return Books
3. Add Books
4. Display Available Books
5. Search For Books
6. View Top Users
7. Exit System


How can we help? Please enter the number:  2
Please enter the ISBN of the book you would like to return:  2233445566


2024-12-11 21:39:01 - Return attempt for ISBN 2233445566.
Book 'War and Peace' is successfully returned by Kelly.

Welcome to City Library System!
1. Borrow Books
2. Return Books
3. Add Books
4. Display Available Books
5. Search For Books
6. View Top Users
7. Exit System


How can we help? Please enter the number:  3
Enter the title of the book:  The Little Prince
Enter the author of the book:  Antoine de Saint-Exupery
Enter the ISBN of the book:  0099887766
Enter the genre of the book:  Classic
Enter the estimated reading time (in hours):  3


2024-12-11 21:42:57 - Adding book 'The Little Prince'.
Book 'The Little Prince' added successfully.

Welcome to City Library System!
1. Borrow Books
2. Return Books
3. Add Books
4. Display Available Books
5. Search For Books
6. View Top Users
7. Exit System


How can we help? Please enter the number:  4
Enter page number to display:  2


2024-12-11 21:43:05 - Displaying available books.

Displaying page 2 of 2:
6. War and Peace,Leo Tolstoy,2233445566,Available,,Historical,12
7. The Alchemist,Paulo Coelho,3344556677,Available,,Philosophy,4
8. Three Little Pigs,Joseph Jacobs,5544332211,Available,,Fairy tale,2
9. Jane Eyre,Charlotte Bronte,1357924680,Available,,Classic,5
10. The Little Prince,Antoine de Saint-Exupery,0099887766,Available,,Classic,3

Page 2 of 2
Previous Page: 1

Welcome to City Library System!
1. Borrow Books
2. Return Books
3. Add Books
4. Display Available Books
5. Search For Books
6. View Top Users
7. Exit System


How can we help? Please enter the number:  5
You would like to search by (title/author/status/genre):  title
